In [ ]:
%reload_ext autoreload
%autoreload 2
%reload_ext dotenv
%dotenv

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from langchain_core.language_models import BaseChatModel
from datasets import Dataset

pd.set_option("display.max_colwidth", None)

# Visualize RAG workflow

<img src="rag_workflow.png" height="1000">


### Load your knowledge base

In [ ]:
import datasets

ds = datasets.load_dataset("A-Roucher/huggingface_doc", split="train")

# 1. Setup evaluation pipeline
In this part, we build a synthetic dataset of questions and associated contexts.

The idea is to randomly get elements from our knowledge base, and ask a LLM to generate questions based on these documents.

In [ ]:
LOAD_PREGENERATED_EVALUATION_DATASET = True

if LOAD_PREGENERATED_EVALUATION_DATASET:
    eval_dataset = datasets.load_dataset("A-Roucher/huggingface_doc_qa_eval")

### Prepare source documents

We use Langchain's `RecursiveCharacterTextSplitter`, which makes efficient use of code language detection to make better splits.

In [ ]:
if not LOAD_PREGENERATED_EVALUATION_DATASET:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.docstore.document import Document as LangchainDocument

    langchain_docs = [
        LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
        for doc in tqdm(ds)
    ]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=200,
        add_start_index=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in langchain_docs:
        docs_processed += text_splitter.split_documents([doc])

### Setup chains for question generation

In [ ]:
if not LOAD_PREGENERATED_EVALUATION_DATASET:
    from langchain.prompts import ChatPromptTemplate
    from langchain.chat_models import ChatOpenAI

    QA_generation_prompt = """
    Your task is to write a factoid question and an answer given a context.
    Your factoid question should be answerable with a specific, concise piece of factual information from the context.
    Your factoid question should be formulated in the same style as questions users could ask in a search engine.
    This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

    Provide your answer as follows:

    Output:::
    Factoid question: (your factoid question)
    Answer: (your answer to the factoid question)

    Now here is the context.

    Context: {context}\n
    Output:::"""

    question_relatedness_critique_prompt = """
    You will be given a context and a question.
    Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
    Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

    Provide your answer as follows:

    Answer:::
    Evaluation: (your rationale for the rating)
    Total rating: (your rating)

    Now here are the question and context.

    Question: {question}\n
    Context: {context}\n
    Answer::: """

    question_relevance_critique_prompt = """
    You will be given a question.
    Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
    Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

    Rating must be minimum, i.e. 1, if the questions refers to a particular setting, like 'in the context' or 'in the document'.

    Provide your answer as follows:

    Answer:::
    Evaluation: (your rationale for the rating)
    Total rating: (your rating)

    Now here is the question.

    Question: {question}\n
    Answer::: """

    question_standalone_critique_prompt = """
    You will be given a question.
    Your task is to provide a 'total rating' representing how context-independant this question is.
    Give your answer on a scale of 1 to 5, where 1 means that the question only makes sense in a specific context, and 5 means that the question makes sense by itself.
    For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
    The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

    Provide your answer as follows:

    Answer:::
    Evaluation: (your rationale for the rating)
    Total rating: (your rating)

    Now here is the question.

    Question: {question}\n
    Answer::: """

    chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.2)
    QA_generation_prompt = ChatPromptTemplate.from_template(QA_generation_prompt)
    QA_generation_chain = QA_generation_prompt | chat_model

    question_relatedness_critique_prompt = ChatPromptTemplate.from_template(
        question_relatedness_critique_prompt
    )
    question_relatedness_critique_chain = (
        question_relatedness_critique_prompt | chat_model
    )

    question_relevance_critique_prompt = ChatPromptTemplate.from_template(
        question_relevance_critique_prompt
    )
    question_relevance_critique_chain = question_relevance_critique_prompt | chat_model

    question_standalone_critique_prompt = ChatPromptTemplate.from_template(
        question_standalone_critique_prompt
    )
    question_standalone_critique_chain = (
        question_standalone_critique_prompt | chat_model
    )

In [ ]:
import random

if not LOAD_PREGENERATED_EVALUATION_DATASET:
    N_GENERATIONS = 30

    print(f"Generating {N_GENERATIONS} QA couples...")
    outputs = []
    for context in tqdm(random.sample(langchain_docs, N_GENERATIONS)):
        # Generate QA couple
        output_QA_couple = QA_generation_chain.invoke(
            {"context": context.page_content}
        ).content
        try:
            question = output_QA_couple.split("Factoid question: ")[1].split(
                "Answer: "
            )[0]
            answer = output_QA_couple.split("Answer: ")[1]
            outputs.append(
                {
                    "context": context.page_content,
                    "question": question,
                    "answer": answer,
                    "source_doc": context.metadata["source"],
                }
            )
        except:
            continue

In [ ]:
if not LOAD_PREGENERATED_EVALUATION_DATASET:
    print("Generating critique for each QA couple...")
    for output in tqdm(outputs):
        # Critique the generated QA couple
        question_relatedness_evaluation = question_relatedness_critique_chain.invoke(
            {"context": output["context"], "question": output["question"]}
        ).content
        question_relevance_evaluation = question_relevance_critique_chain.invoke(
            {"question": output["question"]}
        ).content
        question_standalone_evaluation = question_standalone_critique_chain.invoke(
            {"question": output["question"]}
        ).content

        try:
            relatedness_score = int(
                question_relatedness_evaluation.split("Total rating: ")[1][0]
            )
            relatedness_eval = question_relatedness_evaluation.split("Total rating: ")[
                0
            ].split("Evaluation: ")[1]
            relevance_score = int(
                question_relevance_evaluation.split("Total rating: ")[1][0]
            )
            relevance_eval = question_relevance_evaluation.split("Total rating: ")[
                0
            ].split("Evaluation: ")[1]
            standalone_score = int(
                question_standalone_evaluation.split("Total rating: ")[1][0]
            )
            standalone_eval = question_standalone_evaluation.split("Total rating: ")[
                0
            ].split("Evaluation: ")[1]
            output.update(
                {
                    "relatedness_score": relatedness_score,
                    "relatedness_eval": relatedness_eval,
                    "relevance_score": relevance_score,
                    "relevance_eval": relevance_eval,
                    "standalone_score": standalone_score,
                    "standalone_eval": standalone_eval,
                }
            )
        except:
            continue

Filter down questions on the chosen criteria:

In [ ]:
if not LOAD_PREGENERATED_EVALUATION_DATASET:
    import pandas as pd

    pd.set_option("display.max_colwidth", None)

    generated_questions = pd.DataFrame.from_dict(outputs)
    display(generated_questions.head())
    generated_questions = generated_questions.loc[
        (generated_questions["relatedness_score"] >= 4)
        & (generated_questions["relevance_score"] >= 3)
        & (generated_questions["standalone_score"] >= 4)
    ]
    generated_questions.to_excel("qa_eval.xlsx")
    display(generated_questions.head())

    eval_dataset = datasets.Dataset.from_pandas(
        generated_questions, split="train", preserve_index=False
    )

# 2. Build RAG System

In [ ]:
RAG_SETTINGS = {
    "retriever": {"chunk_size": 1000, "use_instruct_embeddings": True},
    "reader": {
        "model_id": "facebook/dpr-reader-single-nq-base",
        "rerank_results": True,
    },
    "pipeline": {"FLARE": False},
}

## 2.1. Retriever - embeddings
Here we use Langchain vector databases since it offers a convenient FAISS index and allows us to keep document metadata throughout the processing.

Options included:

- Tune the size of chunks retrieved
- Change the embedding model

More could be considered:
- Try another chunking method, like semantic chunking
- Change the index used (here, FAISS)

### Preprocessing

In this part, we split the documents from our knowledge base into smaller chunks which will be the snippets that will support our answer.

The goal is to have semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large in order to be centered around a key information.

[This space](https://huggingface.co/spaces/A-Roucher/chunk_visualizer) lets you visualize how different chunking options affect the chunks you get, to help you tune this step.

Options:
- split respecting sentence boundaries
- semantic splits


Depending on your embedding model, chunks might be split at 512 tokens: so you may want to visualize the length.

In [ ]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


def split_documents(chunk_size: int, knowledge_base: List[LangchainDocument]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    return docs_processed


docs_processed = split_documents(1000, RAW_KNOWLEDGE_BASE)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

lengths = [len(tokenizer.encode(doc.page_content)) for doc in docs_processed]

In [ ]:
pd.Series(lengths).hist()

⬇️🚨 The cell below takes 15min to run on A10G for chunk size 1000, and non-instruct embeddings!

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import (
    HuggingFaceInstructEmbeddings,
    HuggingFaceEmbeddings,
    HuggingFaceBgeEmbeddings,
)
from langchain_community.vectorstores.utils import DistanceStrategy

import os


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    use_instruct_embeddings: Optional[bool] = False,
):
    # load embedding_model
    if use_instruct_embeddings:
        model_name = "thenlper/gte-small"
        embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={"device": "cuda"},
            encode_kwargs={
                "normalize_embeddings": True
            },  # set True to compute cosine similarity
        )

    else:
        model_name = "BAAI/bge-base-en-v1.5"
        embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            multi_process=True,
            model_kwargs={"device": "cuda"},
            encode_kwargs={
                "normalize_embeddings": True
            },  # set True to compute cosine similarity
        )

    # Check if embeddings already exist on disk
    index_name = f"index_chunk:{chunk_size}_instruct:{use_instruct_embeddings}"
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
        )

    else:
        print("Index not found, generating it...")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=100,
            add_start_index=True,
            separators=["\n\n", "\n", ".", " ", ""],
        )
        docs_processed = []
        for doc in langchain_docs:
            docs_processed += text_splitter.split_documents([doc])

        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

#### Test retrieval:

In [ ]:
KNOWLEDGE_INDEX = load_embeddings(
    RAW_KNOWLEDGE_BASE,
    chunk_size=1000,
    use_instruct_embeddings=False,
)
print("\nStarting retrieval:")
docs = KNOWLEDGE_INDEX.similarity_search(query="How to create a pipeline object?", k=5)
print(docs[0].page_content)
print(docs[0].metadata)

## 2.2 Reader - LLM
Here we tried the following options to improve results
- Turn on/off reranking
- Change the reader model

Many options could be considered here to further improve the results
- Add few-shot prompting: this was cancelled becaue it would add a lot of noise in the prompt.
- add prompt compression to keep only the relevant parts of the prompt.
- further extensions:
    - cite source
    - make conversational

In [ ]:
prompt_template = """
<|system|>
Using the information contained in the context, 
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
  </s>
<|assistant|>
"""

In [ ]:
from transformers import pipeline

llm = pipeline(
    "text-generation", model="HuggingFaceH4/zephyr-7b-beta", max_new_tokens=3500
)

llm("Ok,")

In [ ]:
import requests
import os

API_URL = "https://ytjpei7t003tedav.us-east-1.aws.endpoints.huggingface.cloud"

headers = {
    "Authorization": f"Bearer {os.getenv('HF_TOKEN')}",
    "Content-Type": "application/json",
}


def get_llm_answer(question):
    payload = {
        "inputs": question,
        "max_new_tokens": 3000,
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [ ]:
from ragatouille import RAGPretrainedModel


def apply_reranking(
    question: str,
    documents: List[LangchainDocument],
    reranker: RAGPretrainedModel,
    num_docs_final: Optional[int] = 5,
) -> List[LangchainDocument]:
    # keep only the top k documents
    documents_sorted_in_order = reranker.rerank(question, documents, k=num_docs_final)

    # Even and odd documents are ordered in opposite directions to put the most relevant documents at the beginning and end
    documents_even = documents_sorted_in_order[::2]
    documents_odd = documents_sorted_in_order[1::2]
    documents = documents_even + documents_odd[::-1]
    return documents

In [ ]:
def answer_with_rag(
    question: str,
    llm: BaseChatModel,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]

    # Optionally rerank results
    if reranker:
        apply_reranking(
            question, relevant_docs, reranker, num_docs_final=num_docs_final
        )

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"{str(i)}: " + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = prompt_template.format(context=context, question=question)

    # Redact an answer
    output = llm(final_prompt)
    try:
        answer = output[0]["generated_text"]
    except:
        # Retry with shorter contect
        context = "\nExtracted documents:\n"
        context += "".join(
            [f"{str(i)}: " + doc for i, doc in enumerate(relevant_docs[:-1])]
        )

        final_prompt = prompt_template.format(context=context, question=question)

        # Redact an answer
        output = llm(final_prompt)
        answer = output[0]["generated_text"]

    return answer, relevant_docs

## 2.3 Test
How does our RAG pipeline perform?

In [ ]:
question = "how to create a pipeline object?"
answer, relevant_docs = answer_with_rag(
    question, get_llm_answer, KNOWLEDGE_INDEX, reranker=None
)

In [ ]:
print("Answer:")
print(f"{answer}")
print("\n\nSource documents:")
for doc in relevant_docs:
    print(doc)

# 3. Benchmarking the final system on your evaluation set

In [ ]:
import json


def run_rag_tests(
    eval_dataset: Dataset,
    llm: BaseChatModel,
    knowledge_index: FAISS,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
):
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []
    for example in tqdm(eval_dataset["train"]):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question, llm, knowledge_index, reranker=reranker
        )
        if verbose:
            print(":::::::")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        outputs.append(
            {
                "question": question,
                "true_answer": example["answer"],
                "source_doc": example["source_doc"],
                "generated_answer": answer,
                "retrieved_docs": [doc for doc in relevant_docs],
            }
        )

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [ ]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [ ]:
from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
evaluator_name = "GPT4"


def evaluate_answers(
    answer_path: str,
    eval_chat_model: BaseChatModel,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    try:  # load previous generations if they exist
        with open(answer_path, "r") as f:
            answers = json.load(f)
    except:
        answers = []

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [
            item.strip() for item in eval_result.content.split("[RESULT]")
        ]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

In [ ]:
RAG_SETTINGS = {}

for chunk_size in [500]:
    for use_instruct_embeddings in [True]:
        for rerank in [True, False]:
            RAG_SETTINGS[
                f"chunk:{chunk_size}_instruct:{use_instruct_embeddings}_rerank:{rerank}_reader:mistral-7b"
            ] = {
                "retriever": {
                    "chunk_size": chunk_size,
                    "use_instruct_embeddings": use_instruct_embeddings,
                },
                "reader": {
                    "rerank_results": rerank,
                },
            }

In [ ]:
get_llm_answer("ok")

In [ ]:
for settings_name, settings in RAG_SETTINGS.items():
    print(f"Running evaluation for {settings_name}:")
    output_file_name = f"./output/rag_{settings_name}.json"

    print("Loading knowledge base embeddings...")
    knowledge_index = load_embeddings(
        RAW_KNOWLEDGE_BASE,
        chunk_size=settings["retriever"]["chunk_size"],
        use_instruct_embeddings=settings["retriever"]["use_instruct_embeddings"],
    )

    print("Running RAG...")
    if settings["reader"]["rerank_results"]:
        reranker = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
    else:
        reranker = None
    run_rag_tests(
        eval_dataset=eval_dataset,
        llm=get_llm_answer,
        knowledge_index=knowledge_index,
        output_file=output_file_name,
        reranker=reranker,
        verbose=False,
    )

    print("Running evaluation...")
    evaluate_answers(
        output_file_name,
        eval_chat_model,
        evaluator_name,
        evaluation_prompt_template,
    )

In [ ]:
import glob

results = []
for file in glob.glob("output/*.json"):
    print(file)
    with open(file, "r") as f:
        sub_df = pd.read_json(f)
        sub_df["settings"] = file.split("rag_")[1].split(".json")[0]
        results.append(sub_df)

results_df = pd.concat(results)

In [ ]:
def interpret_result(x):
    try:
        return (int(x) - 1) / 4
    except:
        return 0


results_df["normalized_score"] = results_df["eval_score_GPT4"].apply(interpret_result)

In [ ]:
results_df.groupby("settings")["normalized_score"].mean().sort_values()